In [1]:
'065676'

'065676'

# Imports

In [1]:
import backtrader
from helpers.investing import *
from helpers.journal_handler import *

# Defaults

In [2]:
pd.set_option('display.max_rows', 100)

private_key = join(expanduser('~'),'Documents','client_secret.json')
journal = JournalHandler(private_key).get_journal()

In = Investing()

# Work

In [5]:
df = In.show_full_stats(10000,100.10, diff = None, nifty='nifty_200')

df = df[(df['Expected Change %'] <=10) & (df['RSI']<70) & (df['Ichi']>=1)]
df

/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 105.31 for you to afford SIEMENS
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 151.35 for you to afford HDFC
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 162.11 for you to afford BRITANNIA
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 124.23 for you to afford ASIANPAINT
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 144.31 for you to afford TCS
  warnings.warn(f"Risk should be atleast {r} for 

,DATE,OPEN,HIGH,LOW,LTP,CLOSE,52W H,52W L,SYMBOL,Diff,Rising,Ichi,RSI,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
7,2021-08-03,229.00,230.35,225.10,228.95,229.55,261.25,107.55,APOLLOTYRE,0.12,True,1,53.777568,6.139286,Unknown,Unknown,Hammer,Nifty 200,6.79,94.19
12,2021-08-03,180.15,182.80,178.00,181.15,181.15,213.20,75.60,NMDC,0.16,False,1,56.660356,5.632143,Unknown,Unknown,Hammer,Nifty 100,7.46,95.73
32,2021-08-03,207.50,210.90,206.45,209.75,209.90,239.20,163.35,ITC,0.17,False,1,57.509578,3.871429,Unknown,Unknown,Unknown,Nifty 50,6.45,95.54
15,2021-08-03,86.05,89.40,86.05,89.05,89.15,92.50,45.35,FEDERALBNK,0.35,True,2,58.578830,2.282143,Unknown,Bullish Engulfing,Inverted Hammer,Nifty 200,8.76,98.22
34,2021-08-03,156.00,157.70,154.80,156.75,157.15,174.60,82.25,CANBK,1.91,False,2,59.825931,4.117857,Unknown,Unknown,Hammer,Nifty 200,5.79,96.16
31,2021-08-03,147.20,148.90,146.15,147.90,148.20,157.30,76.35,INDHOTEL,3.17,True,1,54.793881,4.271429,Unknown,Unknown,Hammer,Nifty 200,6.85,97.26
17,2021-08-03,434.95,447.90,433.60,446.00,446.50,447.90,175.50,SBIN,6.02,True,2,62.602390,9.117857,Unknown,Unknown,Bullish,Nifty 50,8.46,95.02
28,2021-08-03,164.60,166.85,163.30,166.80,166.00,177.40,83.50,OIL,6.43,True,2,60.003124,4.675000,Reverse V Pattern,Unknown,Unknown,Nifty 200,5.54,97.43
11,2021-08-03,987.00,1025.00,986.65,1022.45,1022.05,1119.50,483.25,INDUSINDBK,8.06,True,1,54.987966,24.682143,Unknown,Bullish Engulfing,Inverted Hammer,Nifty 50,8.98,92.37
42,2021-08-03,1186.00,1198.00,1181.80,1188.00,1188.95,1284.40,779.65,MGL,11.07,False,1,58.892329,30.617857,Unknown,Unknown,Hammer,Nifty 200,5.96,71.63


In [8]:
df = In.open_downloaded_stock('SBIN')
df = In.open_live_stock_data('SBIN')
In.get_particulars('SBIN',budget=10000,max_loss_capacity=300,risk_to_reward_ratio=2)

/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:197: UserWarning: You are risking 3.0% of your total PORTFOLIO. Going Like this, you'll lose Rs 4500 in 15 Trades. Try keeping it less than 1.1% @ Rs 110.0.
  warnings.warn(f"You are risking {round(max_loss_capacity/budget,2)*100}% of your total PORTFOLIO. Going Like this, you'll lose Rs {max_loss_capacity*15} in 15 Trades. Try keeping it less than 1.1% @ Rs {round(budget*0.011,2)}.")


{'Buying Price': 449.38,
 'Stop-Loss %': 4.23,
 'Target %': 8.46,
 'Quantity': 15.0,
 'Stop-Loss Price': 430.37506,
 'Trigger Price': 487.38,
 'Risk Per Share': 19.0,
 'Profit Per Share': 38.01,
 'Max loss on this config': 285.05,
 'Max Gain on this config': 570.09,
 'Price To Profit Ratio': 11.82,
 'Index': 'Nifty 50'}

In [16]:
df = AnalyseStocks().get_MA(In.open_downloaded_stock('NMDC'), 200, simple=True)
df['44-MA'].sort_index(ascending=True).diff().head(20)

0          NaN
1     0.005682
2     0.045455
3    -0.044318
4    -0.095455
5     0.065909
6     0.161364
7     0.182955
8     0.061364
9     0.070455
10    0.290909
11    0.289773
12    0.179545
13    0.297727
14    0.702273
15    0.571591
16    0.780682
17    0.390909
18    0.075000
19   -0.020455
Name: 44-MA, dtype: float64

In [20]:
df['44-MA'].pct_change(1).head(100)

0          NaN
1     0.000032
2     0.000255
3    -0.000249
4    -0.000536
5     0.000370
6     0.000906
7     0.001026
8     0.000344
9     0.000395
10    0.001629
11    0.001620
12    0.001002
13    0.001660
14    0.003909
15    0.003169
16    0.004314
17    0.002151
18    0.000412
19   -0.000112
20   -0.001466
21   -0.002030
22   -0.003405
23   -0.003825
24   -0.004609
25   -0.004447
26   -0.005020
27   -0.005001
28   -0.004441
29   -0.005319
30   -0.006166
31   -0.005531
32   -0.004846
33   -0.005464
34   -0.005912
35   -0.004759
36   -0.005580
37   -0.005975
38   -0.005468
39   -0.005246
40   -0.005452
41   -0.006475
42   -0.008432
43   -0.008238
44   -0.008363
45   -0.007750
46   -0.007201
47   -0.007376
48   -0.006579
49   -0.006572
50   -0.007168
51   -0.006938
52   -0.006036
53   -0.006713
54   -0.007849
55   -0.008560
56   -0.007448
57   -0.008202
58   -0.010234
59   -0.009557
60   -0.010559
61   -0.008583
62   -0.007062
63   -0.006449
64   -0.005600
65   -0.005996
66   -0.00